### Notes
I would normally use a configuration file, environment variables, or command-line arguments for the variables in the "Credentials" cell. However, given the format of the exercise, I feel just having the variables integrated in the notebook in their own cell is the simplest for everyone involved.

In [1]:
# Imports
import os, json
import pandas
import boto3
from enum import Enum
from botocore.exceptions import ClientError, ReadTimeoutError, ConnectTimeoutError

In [2]:
# Credentials
aws_access_key=""
aws_secret_key=""
aws_s3_bucket=""
postgres_host=""
postgres_port=5432
postgres_user=""
postgres_password=""
postgres_database="postgres"
postgres_table=""

In [3]:
class MIND_S3:
    """Holds AWS S3 credentials and session as well as performs actions against S3."""
    aws_access_key = None
    aws_secret_key = None
    s3_client = None

    # Defaults
    max_keys = 1000


    def __init__(self,access_key: str,secret_key: str) -> None:
        """
        Initialize the S3 client.

        access_key: Your AWS Access Key ID

        secret_key: Your cooresponding AWS Secret Key
        """
        self.aws_access_key = access_key
        self.aws_secret_key = secret_key
        self.__login()

    def __login(self,retries=0):
        """Attempt to login to AWS S3 using the saved credentials."""
        try:
            self.s3_client = boto3.client(
                "s3",
                aws_access_key_id = self.aws_access_key,
                aws_secret_access_key = self.aws_secret_key
            )
        except TimeoutError or ReadTimeoutError or ConnectTimeoutError as e:
            if(retries > 2):
                raise e
            else:
                self.__login(retries+1)
        except Exception as e:
            raise e
        
    def list_all_objects(self,bucket: str,prefix="") -> list:
        """
        Paginate through all of the objects in a given bucket which contain a given prefix.

        Warning: This can become an expensive process on large buckets.

        Returns: list(object) -- Objects in format specified by boto3 list_objects_v2()

        Args:

        bucket -- The S3 bucket to perform the list on.

        prefix -- The prefix to use when filtering results.
        """
        objects = []
        response = self.s3_client.list_objects_v2(
            Bucket=bucket,
            MaxKeys=self.max_keys,
            Prefix=prefix
        )

        if("Contents" in response):
            objects.extend(response["Contents"])
        
        while ("NextContinuationToken" in response):
            response = self.s3_client.list_objects_v2(
                Bucket=bucket,
                MaxKeys=self.max_keys,
                Prefix=prefix,
                ContinuationToken=response["NextContinuationToken"]
            )
            objects.extend(response["Contents"])

        return objects
    
    def pull_objects(self,bucket: str, destination: str, objects: list) -> list:
        """
        Attempts to download the given list of objects from the given bucket and place then in a directory on
        the local filesystem.

        Returns: list(str) -- The the paths to each of the retrieved files.

        Args:

        bucket -- The S3 bucket to pull from.

        destination -- The target directory to store the retrieved objects. (Will be created if it doesn't already exist.)

        objects -- A list of objects to pull. Must be in format [{"Key":"object_key"}]. MIND_S3.list_all_objects() returns this format.
        """
        files = []
        os.makedirs(destination,exist_ok=True)
        for item in objects:
            try:
                obj = self.s3_client.get_object(
                    Bucket=bucket,
                    Key=item["Key"]
                )
                if("Body" in obj):
                    file_path = "%s/%s"%(destination,item["Key"])
                    object_directory = os.path.dirname(file_path)
                    os.makedirs(object_directory,exist_ok=True)

                    with open(file_path,'wb') as file:
                        file.write(obj["Body"].read())

                    files.append(file_path)
            except ClientError as e:
                if e.response['Error']['Code'] == 'NoSuchKey':
                    print("Object %s not found."%(item["Key"]))

        return files

In [4]:
class Result(Enum):
    """The possible results of a game."""
    WIN=1.0
    LOSS=0.0

In [5]:
def parseCSVFile(file_name: str) -> pandas.DataFrame:
    """
    Loads in a CSV file and creates a pandas DataFrame from its contents.

    Args:

    file_name -- path to the CSV file which should be parsed.
    """
    frame = pandas.read_csv(file_name)
    if("Yards" in frame):
        file_base_name = os.path.basename(file_name)
        player_name = file_base_name.split("_")[0].capitalize()
        frame = frame.rename(columns={
            "Yards":"%s Yards"%(player_name),
            "TD":"%s TD"%(player_name),
        })
        return frame
    if("Result" in frame):
        frame["Result"] = frame["Result"].replace(Result.WIN.value, "Win").replace(Result.LOSS.value, "Loss")
        return frame
    
    return frame

In [6]:
# Fetch the CSV files from S3
s3 = MIND_S3(aws_access_key,aws_secret_key)
objects = s3.list_all_objects(aws_s3_bucket)
files = s3.pull_objects(aws_s3_bucket,"./temp",objects)

In [7]:
# Parse and transform data from CSV files.
resultFrame = None
for file_name in files:
    parsed_frame = parseCSVFile(file_name)
    if(resultFrame is None):
        resultFrame = parsed_frame
    elif(parsed_frame is not None):
        resultFrame = resultFrame.merge(parsed_frame,how="outer")